<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
using Pkg; Pkg.activate(".")

  Activating project at `C:\Users\lukas\Desktop\AMO\Team_assignment_AMO`


In [2]:
using JuMP,HiGHS,DataFrames

In [36]:
nP=1:3                                  #number of generation units
nW=1:1                                  #number of stochastic generation units
nt=1:4                                  #number of time intervalls 
nS=1:2                                  #number of scenarios
C=[100, 80, 50]                         #unit cost of generation
C_RU=[20, 10, 15]                       #cost of up reserve commitment
C_RD=[25, 15, 15]                       #cost of down reserve commitment
C_U=[10, 15, 20]                        #cost of up reserve commitment
C_D=[30, 10, 15]                        #cost of down reserve commitment
pi=[0.7, 0.3]                           #probabilities of the scenarios
V_LOL=[20, 25]                          #value of the lost load
C_cur=[15]                              #cost of curtailment at turbine
L=[150, 60]                             #loads
W_realized = 70 .+ 15 * randn(2, 4)     #create random scenarios for four time points
P_max=[70, 80, 150]                     #maximum production capacity

3-element Vector{Int64}:
  70
  80
 150

In [37]:
data= Dict(
    :nP => nP,
    :nW => nW,
    :nt => nt,
    :nS => nS,
    :C => C,
    :C_RU => C_RU,
    :C_RD => C_RD,
    :P_max => P_max,
    :C_U => C_U,
    :C_D => C_D,
    :pi => pi,
    :V_LOL => V_LOL,
    :C_cur => C_cur,
    :L => L,
    :W_realized => W_realized,
    :P_max => P_max
    )

Dict{Symbol, AbstractArray} with 15 entries:
  :C_RU       => [20, 10, 15]
  :W_realized => [82.6716 93.7733 75.6023; 67.1494 57.0342 83.8781]
  :C_U        => [10, 15, 20]
  :nW         => 1:1
  :nS         => 1:2
  :C_D        => [30, 10, 15]
  :C_RD       => [25, 15, 15]
  :V_LOL      => [20, 25]
  :P_max      => [70, 80, 150]
  :C_cur      => [15]
  :pi         => [0.7, 0.3]
  :nP         => 1:3
  :nt         => 1:4
  :L          => [150, 60]
  :C          => [100, 80, 50]

In [116]:
model=Model(HiGHS.Optimizer);

In [117]:
function init_variables(model::JuMP.Model)
    @variable(model, P[p in data[:nP]] >=0)
    @variable(model, R_U[p in data[:nP]] >=0)
    @variable(model, R_D[p in data[:nP]] >=0)
    @variable(model, r_U[p in data[:nP], s in data[:nS], t in data[:nt]] >=0)
    @variable(model, r_D[p in data[:nP], s in data[:nS], t in data[:nt]] >=0)
    @variable(model, L_Shed[p in data[:nP], s in data[:nS], t in data[:nt]] >=0)
    @variable(model, W_spill[p in data[:nP], s in data[:nS], t in data[:nt]] >=0)
    @variable(model, W_s[w in data[:nW]] >=0)
    
    return P, R_U, R_D, r_U, r_D, L_Shed, W_spill, W_s
end

P, R_U, R_D, r_U, r_D, L_Shed, W_spill, W_s=init_variables(model);

In [118]:
function init_constraints(model::JuMP.Model, data::Dict, P, R_U, R_D, r_U, r_D, L_Shed, W_spill, W_s)
    @constraints(model, begin 
            c1[p in data[:nP]], P[p]+ R_U[p]<= P_max[p]
            c2[p in data[:nP]], P[p]- R_D[p]>= 0
            c3, sum(P) + sum(W_s) - sum(L) == 0
            c4[s in data[:nS], t in data[:nt]], sum()
        
    end)
end
init_constraints(model, data, P, R_U, R_D, r_U, r_D, L_Shed, W_spill, W_s);

In [119]:
print(model)

In [ ]:
function init_obj_function()
    
end